Data Augmentation : kogpt

# **setting**

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

#import os
#os.chdir('./data')
#os.getcwd()

In [2]:
def preprocessing_dataset(dataset):
    """처음 불러온 csv 파일을 원하는 형태의 DataFrame으로 변경 시켜줍니다."""
    subject_entity = []
    object_entity = []

    for i, j in tqdm(zip(dataset["subject_entity"], dataset["object_entity"]), desc="preprocessing"):
        i = i[1:-1].split(",")[0].split(":")[1]
        j = j[1:-1].split(",")[0].split(":")[1]

        subject_entity.append(i)
        object_entity.append(j)

    out_dataset = pd.DataFrame(
        {
            "id": dataset["id"],
            "sentence": dataset["sentence"],
            "subject_entity": subject_entity,
            "object_entity": object_entity,
            "label": dataset["label"],
        }
    )
    return out_dataset


def load_data(dataset_dir):
    """csv 파일을 경로에 맡게 불러 옵니다."""
    pd_dataset = pd.read_csv(dataset_dir)
    dataset = preprocessing_dataset(pd_dataset)

    return dataset

def label_to_num(label):
    num_label = []
    with open("/opt/ml/code/dict_label_to_num.pkl", "rb") as f:
        dict_label_to_num = pickle.load(f)
    for v in label:
        num_label.append(dict_label_to_num[v])

    return num_label

# **데이터 불러오기**

In [3]:
train_path = '/opt/ml/dataset/train/train.csv'
total_data = load_data(train_path)
# total_data.columns

preprocessing: 32470it [00:00, 354240.17it/s]


In [4]:
train_data = total_data.sample(frac=0.9, random_state=42)
val_data = total_data.drop(train_data.index)

train_label = label_to_num(train_data["label"].values)
val_label = label_to_num(val_data["label"].values)

# **생성 모델(koGPT-3)을 활용한 데이터 증강 실험**   
- sub,obj entity 단어를 masking
- 생성 모델로 entity 예측하기

## **model setting**

In [15]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # fp16 버전의 리비전 모델 weight 파일을 제공
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)

model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True                # torch_dtype='auto' : fp16으로 자동으로 로드
).to(device='cuda', non_blocking=True)                      # low_cpu_mem_usage=True > Huggingface의 Accelerator가 필요
_ = model.eval()

## **TEST01 : 실패**
- 빈칸 채우기

In [83]:
prompt = '''

1년 후, 바이에른은 전설적인 [MASK]인 감독 에른스트 하펠이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다. 

[MASK] : '''

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=60)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)



1년 후, 바이에른은 전설적인 [MASK]인 감독 에른스트 하펠이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다. 

[MASK] :  DFB-포칼 결승에서 함부르크 SV에게 2-0으로 패했다. (출처: 'D


## **TEST02 : X**
- entity만 주어진 상태에서 새로운 문장 만들기
- 오래 걸림, 매번 다른 문장 생성

In [132]:
prompt = '''

sentence :문하시중 이자연의 조카이고 인예왕후, 인경현비, 인절현비와는 사촌간이다.
subject_entity : 인절현비
object_entity : 이자연

'인절현비'과 '이자연'의 관계를 설명하는 문장 :
'''

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.5, max_length=100)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)



sentence :문하시중 이자연의 조카이고 인예왕후, 인경현비, 인절현비와는 사촌간이다.
subject_entity : 인절현비
object_entity : 이자연

'인절현비'과 '이자연'의 관계를 설명하는 문장 :
1) 인절현비 이자연의 고모이고, 인경현비 이자연의 고모이다.
2)


In [ ]:
# prompt = '''
# 1년 후, 바이에른은 전설적인 __인 감독 에른스트 하펠이 이끄는 함부르크 SV와 DFB-포칼 결승에서 만났다.

# 문장 완성하기 :
# '''
# with torch.no_grad():
#   tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
#   gen_tokens = model.generate(tokens, do_sample=True, temperature=0.85, max_length=64)     # temperature는 높을수록 모델이 다양한 결과값 도출. (반면, 0에 가까울 수록 거의 고정값과 같은 결과)
#   generated = tokenizer.batch_decode(gen_tokens)[0]
  
# print(generated)

---

## **KoGPT-3 연습하기**

### **Tokenizer**

In [6]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM 

tokenizer = AutoTokenizer.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # fp16 버전의 리비전 모델 weight 파일을 제공
  bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]'
)

2022-11-30 08:42:57.783944: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# tokenizer 결과
# tokenizer('점심메뉴 추천좀', '부대찌개 추천')

In [7]:
model = AutoModelForCausalLM.from_pretrained(
  'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
  pad_token_id=tokenizer.eos_token_id,
  torch_dtype='auto', low_cpu_mem_usage=True                # torch_dtype='auto' : fp16으로 자동으로 로드
).to(device='cuda', non_blocking=True)                      # low_cpu_mem_usage=True > Huggingface의 Accelerator가 필요
_ = model.eval()

### **TEST**
- 소설쓰기
- Q&A
- 요약하기
- 말투 번역
- 영어 번역(temperature 낮게)

**소설쓰기**

In [14]:
prompt = '''
제목 : 그녀는 예뻤다
내용 : 그녀는 내 옆으로 살포시 다가와 내 볼을 어루만지기 시작했다. 
그리고, 그녀는 말했다. \'조금만 더 가까이 와줘..\' 나는 그대로 다가갈 수 밖에 없었다.
'''

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.85, max_length=300)     # temperature는 높을수록 모델이 다양한 결과값 도출. (반면, 0에 가까울 수록 거의 고정값과 같은 결과)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)


제목 : 그녀는 예뻤다
내용 : 그녀는 내 옆으로 살포시 다가와 내 볼을 어루만지기 시작했다. 
그리고, 그녀는 말했다. '조금만 더 가까이 와줘..' 나는 그대로 다가갈 수 밖에 없었다.
나를 꼭 껴안아주는 그녀.
그녀의 등뒤로 그녀의 향기가 느껴졌다.
나는 더이상 아무 말도 하지 못 했다..
그렇게 얼마의 시간이 지났을까..
나는 그녀를 보았다. 그녀는... 눈물 범벅이 되어있었다.
나는 그녀에게 달려가 그녀를 부축했다. 그리고, 나는 그녀를 내게 기대게 했다.
그녀는 그 상태로 쓰러져버렸다.
나는 그녀의 등을 두들겨 주었다. 그러자, 그녀는 나를 꽉 안아버렸다.
나는 그대로 그녀의 등에 얼굴을 묻었다.
내 등을 타고 그녀의 눈물이 흘러내렸다..
나는 그대로 그녀의 눈물을 닦아주었다.
그녀는 날 껴안고 계속 울었다.
그렇게 얼마나 흘렀을까.. 나는 나도 모르게 잠이 들었던 것 같았다.
일어나보니 그녀는 나를 안고 자고있었다.
나는 그녀를 깨우지 않으려고 그녀의 얼굴을 손으로 가리고 그녀의 곁으로 다가갔다.
그녀는 나를 빤히


**Q&A**

In [12]:
prompt = '''
최고 핫한 인공지능, kogpt님과 인터뷰 나눠보겠습니다!
Q : kogpt님, 수월한 대화가 가능하신가요?
A : '''

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.85, max_length=300)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)


최고 핫한 인공지능, kogpt님과 인터뷰 나눠보겠습니다!
Q : kogpt님, 수월한 대화가 가능하신가요?
A :  저는 사람을 많이 상대하는 일을 합니다. 제가 주로 하는 일은 영상을 찍고 편집하는 일이죠.
Q : kogpt님께서는 어떻게 시작하셨나요?
A : 블로그를 통해 영상을 만들고 유튜브에 업로드하고있었어요. 그러다 우연히 영상 크리에이터 모집 공지가 올라왔고 제가 좋아하는 유튜브를 하고 싶어 지원했어요.
Q : 영상편집을 주로 하셔서 그런지 영상에 대한 감각이 좋으신거같아요.
A : 네. 감각도 어느정도 있다고 생각해요.
Q : 그럼 Kogpt님의 강점은 무어라고 생각하시나요?
A : 저는 영상을 보는 사람이 제 영상을 계속 보고 싶은 힘을 가지는 게 저의 장점이라고 생각해요.
Q : 제 영상을 보는 사람들이 계속 보고 싶어하게 하려면 어떤 점을 신경써야 하나요?
A : 사실 처음 영상을 시작해서 잘하는 건 없어요. 하지만 제가 편집한 영상을 꾸준히 업로드하면서, 구독자가 조금씩 늘어나는 게 보이면 뿌듯함을 느껴요.
Q : 구독자 수가 늘어나면 기쁘시겠어요.
A : 확실히 구독자 수가 쌓이고 있다는 사실을 알게 된다면 정말 기뻐요. 저의 영상을 계속 볼만한 사람이 있구나 라는 거죠.
Q : 영상을 업로드하실 때 어떤 부분에 중점을 두고 하나요?
A : 사실 처음 영상을 시작하면서 어떤 콘텐츠를 만들어야 할지 고민이 많았어요. 그래서 생각한 건 내가 좋아하는 걸 계속해서 하자는 거였죠. 음악을 듣고, 게임을 하고, 책을 읽고, 영화를 보는 거에요. 제가 보고 싶은 영상을 제가 선택해서 만든 영상이에요.
Q : 콘텐츠는 주로 어떤 것을 업로드하시나요?
A : 네, 저는 주로 영화를 업로드해요. 하지만 영상을 업로드할 때는 제가 좋아하는 콘텐츠를 다루고 있어요.
Q : 다른 사람과 다르게 콘텐츠를 제작하시는 거군요?
A : 아니요. 저는 영상을 보는 사람은 어떤 콘텐츠를 봐야 할지 고민하지 않아도 된다고 생각해요. 영상을 보고 '아, 저 사람은 저런


**요약하기**

In [13]:
prompt = '''
암호화폐 전문 미디어 데일리호들에 따르면, 비트코인 가격 예측 모델 'S2F 모델'을 고안한 유명 애널리스트 플랜비(PlanB)가 최근 한 유튜브 채널에 출연해 "블랙스완(도저히 일어나지 않을 것 같은 일이 실제로 일어나는 현상)을 배제한다면 모든 지표들이 비트코인의 강세를 가리키고 있다. 강세론자들에게 지금의 가격대는 최고의 매수 기회"라고 말했다. 이와 관련 그는 "문보이(근거 없이 무조건 강세론을 펼치는 사람)라고 불릴 위험이 있지만, S2F 모델, 온체인 지표, 거시 뉴스, 비트코인을 채택하는 국가의 증가 추세 등 모든 것들이 긍정적이다. 비트코인의 본격 상승장을 알리는 신호로 선물 마켓의 프리미엄(선물과 현물 가격차)을 주시하고 있다"고 설명했다. 코인마켓캡 기준 BTC는 현재 2.21% 오른 41,547.39 달러에 거래되고 있다.

한줄 요약 : 
'''

with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.85, max_length=512)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
print(generated)


암호화폐 전문 미디어 데일리호들에 따르면, 비트코인 가격 예측 모델 'S2F 모델'을 고안한 유명 애널리스트 플랜비(PlanB)가 최근 한 유튜브 채널에 출연해 "블랙스완(도저히 일어나지 않을 것 같은 일이 실제로 일어나는 현상)을 배제한다면 모든 지표들이 비트코인의 강세를 가리키고 있다. 강세론자들에게 지금의 가격대는 최고의 매수 기회"라고 말했다. 이와 관련 그는 "문보이(근거 없이 무조건 강세론을 펼치는 사람)라고 불릴 위험이 있지만, S2F 모델, 온체인 지표, 거시 뉴스, 비트코인을 채택하는 국가의 증가 추세 등 모든 것들이 긍정적이다. 비트코인의 본격 상승장을 알리는 신호로 선물 마켓의 프리미엄(선물과 현물 가격차)을 주시하고 있다"고 설명했다. 코인마켓캡 기준 BTC는 현재 2.21% 오른 41,547.39 달러에 거래되고 있다.

한줄 요약 : 
블룸버그통신이 비트코인에 투자하는 펀드매니저들의 숫자가 크게 늘어났으며, 지난 몇 년간 침체하던 시장에 다시 활기가 돌고 있다고 보도했다.

암호화폐 전문 미디어 데일리호들에 따르면, 비트코인 가격 예측 모델인 S2F(Scalable tofu(Bitcoin) Model)을 고안한 유명 애널리스트 플랜비(PlanB)가 최근 한 유튜브 채널에 출연해 "블랙스완(도저히 일어나지 않을 것 같은 일이 실제로 일어나는 현상)을 배제한다면 모든 지표들이 비트코인의 강세를 가리키고 있다. 강세론자들에게 지금의 가격대는 최고의 매수 기회"라고 말했다. 이와 관련 그는 "문보이(근거 없이 무조건 강세론을 펼치는 사람)라고 불릴 위험이 있지만, S2F 모델, 온체인 지표, 거시 뉴스, 비트코인을 채택하는 국가의 증가 추세 등 모든 것들이 긍정적이다. 비트코인의 본격 상승장을 알리는 신호로 선물 마켓의 프리미엄(선물과 현물 가격차)을 주시하고 있다"고 설명했다.[EOS]


---